<a href="https://colab.research.google.com/github/peaysararn/Tools/blob/main/Plotly_practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests
import plotly.express as px
import pandas as pd

In [2]:
data = pd.read_csv('https://raw.githubusercontent.com/peaysararn/Tools/main/suicide_rates_1990-2022.csv')
print(data.info())
# print(data.describe())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 118560 entries, 0 to 118559
Data columns (total 18 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0   RegionCode                    118560 non-null  object 
 1   RegionName                    118560 non-null  object 
 2   CountryCode                   118560 non-null  object 
 3   CountryName                   118560 non-null  object 
 4   Year                          118560 non-null  int64  
 5   Sex                           118560 non-null  object 
 6   AgeGroup                      118560 non-null  object 
 7   Generation                    118560 non-null  object 
 8   SuicideCount                  118096 non-null  float64
 9   CauseSpecificDeathPercentage  114271 non-null  float64
 10  DeathRatePer100K              107896 non-null  float64
 11  Population                    112640 non-null  float64
 12  GDP                           111320 non-nul

In [3]:
data.head()

,RegionCode,RegionName,CountryCode,CountryName,Year,Sex,AgeGroup,Generation,SuicideCount,CauseSpecificDeathPercentage,DeathRatePer100K,Population,GDP,GDPPerCapita,GrossNationalIncome,GNIPerCapita,InflationRate,EmploymentPopulationRatio
0,EU,Europe,ALB,Albania,1992,Male,0-14 years,Generation Alpha,0.0,0.000000,0.000000,3247039.0,652174990.8,200.85222,906184212.3,1740.0,226.005421,45.315
1,EU,Europe,ALB,Albania,1992,Male,0-14 years,Generation Alpha,0.0,0.000000,0.000000,3247039.0,652174990.8,200.85222,906184212.3,1740.0,226.005421,45.315
2,EU,Europe,ALB,Albania,1992,Male,0-14 years,Generation Alpha,0.0,0.000000,0.000000,3247039.0,652174990.8,200.85222,906184212.3,1740.0,226.005421,45.315
3,EU,Europe,ALB,Albania,1992,Male,0-14 years,Generation Alpha,0.0,0.000000,0.000000,3247039.0,652174990.8,200.85222,906184212.3,1740.0,226.005421,45.315
4,EU,Europe,ALB,Albania,1992,Male,15-24 years,Generation Z,5.0,3.401361,3.531073,3247039.0,652174990.8,200.85222,906184212.3,1740.0,226.005421,45.315


In [4]:
df = data.loc[:,["Year","Sex","SuicideCount"]]
df.head()

,Year,Sex,SuicideCount
0,1992,Male,0.0
1,1992,Male,0.0
2,1992,Male,0.0
3,1992,Male,0.0
4,1992,Male,5.0


In [5]:
df.nunique()

Year              33
Sex                3
SuicideCount    1868
dtype: int64

In [6]:
df["Sex"].unique()

array(['Male', 'Female', 'Unknown'], dtype=object)

In [7]:
df_m = df.query('Sex == "Male"')
df_m = df_m.groupby(['Year','Sex'])['SuicideCount'].sum().reset_index()
df_m.head()

,Year,Sex,SuicideCount
0,1990,Male,153515.0
1,1991,Male,155454.0
2,1992,Male,167740.0
3,1993,Male,178430.0
4,1994,Male,188585.0


In [8]:
df_f = df.query('Sex == "Female"')
df_f = df_f = df_f.groupby(['Year','Sex'])['SuicideCount'].sum().reset_index()
df_f.head()

,Year,Sex,SuicideCount
0,1990,Female,53930.0
1,1991,Female,52130.0
2,1992,Female,54223.0
3,1993,Female,53939.0
4,1994,Female,53834.0


In [9]:
df_group = pd.merge(df_m[["Year","SuicideCount"]],df_f[["Year","SuicideCount"]],on = "Year")
df_group = df_group.rename(columns={'SuicideCount_x': 'm_count','SuicideCount_y': 'f_count'})
df_group.head()

,Year,m_count,f_count
0,1990,153515.0,53930.0
1,1991,155454.0,52130.0
2,1992,167740.0,54223.0
3,1993,178430.0,53939.0
4,1994,188585.0,53834.0


In [10]:
df_group["total"] = df_group["m_count"]+df_group["f_count"]
df_group.head()

,Year,m_count,f_count,total
0,1990,153515.0,53930.0,207445.0
1,1991,155454.0,52130.0,207584.0
2,1992,167740.0,54223.0,221963.0
3,1993,178430.0,53939.0,232369.0
4,1994,188585.0,53834.0,242419.0


In [11]:
fig = px.line(df_group, x="Year", y=["m_count","f_count","total"])
fig.show()

In [12]:
df2 = data.loc[:,["Year","SuicideCount","CountryName"]]
df2.head()

,Year,SuicideCount,CountryName
0,1992,0.0,Albania
1,1992,0.0,Albania
2,1992,0.0,Albania
3,1992,0.0,Albania
4,1992,5.0,Albania


In [13]:
df_name = df2.groupby("CountryName")['SuicideCount'].sum().reset_index().sort_values(by='SuicideCount',ascending=False).head()
df_name

,CountryName,SuicideCount
89,Russian Federation,1256440.0
113,United States of America,1164733.0
53,Japan,816888.0
39,Germany,338037.0
86,Republic of Korea,326020.0


In [14]:
df2 = df2[df2['CountryName'].isin(df_name['CountryName'])]
df2 = df2.groupby(['Year',"CountryName"])['SuicideCount'].sum().reset_index()
df2.head()

,Year,CountryName,SuicideCount
0,1990,Germany,13924.0
1,1990,Japan,20088.0
2,1990,Republic of Korea,3159.0
3,1990,Russian Federation,39150.0
4,1990,United States of America,30906.0


In [15]:
fig = px.line(df2, x="Year", y="SuicideCount",color="CountryName")
fig.show()